In [3]:
import pandas as pd

# Cargar dataset multicategoría
df = pd.read_csv("../emg_balanced_classify.csv", keep_default_na=False)

# Inicializar columna para marcar datos sintéticos
df["is_synthetic"] = False

# Gestos requeridos
gestos_a_seleccionar = [
    'index_finger',
    'middle_finger',
    'ring_finger',
    'little_finger',
    'thumb',
    'rest',
    'victory_gesture'
]

# Contenedor para guardar los índices seleccionados
indices_seleccionados = []

# Recorrer cada score (1, 2, 3)
for score in [1, 2, 3]:
    df_score = df[df["gb_score"] == score]
    muestras = []

    for gesto in gestos_a_seleccionar:
        gesto_df = df_score[df_score["gesture"] == gesto]

        if not gesto_df.empty:
            fila = gesto_df.sample(n=1, random_state=42)
        else:
            # Fila sintética si no hay gesto con este score
            print(f"⚠️ Gesto '{gesto}' con score {score} no encontrado. Agregando fila sintética.")
            gesto_base_df = df[df["gesture"] == gesto]
            if not gesto_base_df.empty:
                fila = gesto_base_df.sample(n=1, random_state=42).copy()
                fila["gb_score"] = score
                fila["is_synthetic"] = True
            else:
                print(f"🚫 No hay ninguna fila con el gesto '{gesto}' en todo el dataset.")
                continue

        muestras.append(fila)
        indices_seleccionados.append(fila.index[0])

    # Concatenar y guardar
    df_split = pd.concat(muestras).reset_index(drop=True)
    df_split.to_csv(f"split_score_{score}.csv", index=False)

# Eliminar las filas seleccionadas del dataset original para el archivo final
df_final = df.drop(index=indices_seleccionados).reset_index(drop=True)
df_final.to_csv("emg_classify_final.csv", index=False)

print("✅ Archivos generados:")
print("- split_score_1.csv")
print("- split_score_2.csv")
print("- split_score_3.csv")
print("- emg_classify_final.csv")


⚠️ Gesto 'rest' con score 1 no encontrado. Agregando fila sintética.
✅ Archivos generados:
- split_score_1.csv
- split_score_2.csv
- split_score_3.csv
- emg_classify_final.csv
